In [1]:
import astro
from pathlib import Path
import os
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

In [7]:
%matplotlib widget

In [2]:
cal_dir = Path(os.environ['DATA'], 'DDS', 'Taka', 'Calibration')
mbiasf = Path(cal_dir, 'master_bias_bin1_Mar_2018.fits')
flats_dir = Path(cal_dir, 'Flats')
flatsf = sorted(list(flats_dir.glob('Hflat_Mar_3_2019/HFlat*.fit')))
nframes = len(flatsf)
# load masters bias and dark
mbias = fits.getdata(mbiasf)
# Get reference flat image and calibrate with master bias
# (for comparison, choose same as Pixinsight)
ref = 6
flath = fits.getheader(flatsf[ref])
nx, ny = flath['NAXIS1'], flath['NAXIS2']
flatd = fits.getdata(flatsf[ref]) - mbias

In [3]:
# Reference median for flux equalization prior to pixel rejection
m0 = np.median(flatd)
# Loop over the flat files and calibrate with optimized dark
flat_cube = np.zeros([nframes, ny, nx], dtype=np.float32)
print('Calibrating flat series...')
for i, f in enumerate(flatsf):
    flat_header = fits.getheader(f)
    flat_cube[i] = fits.getdata(f) - mbias
    # At this point, images confirmed to be equivalent to Pixinsight processing
    # # Flux equalization factor
    flux_eq_scale = m0/np.median(flat_cube[i])
    flat_cube[i] *= flux_eq_scale

Calibrating flat series...


In [34]:
def percentile_clip(images, plow=0.01, phigh=0.01):
    print(f'plow = {plow} ; phigh = {phigh}')
    median = np.median(images, axis=0)
    rej_mask1 = median - images > plow * median
    rej_mask2 = images - median > phigh * median
    rej_mask = rej_mask1 | rej_mask2
    return rej_mask, rej_mask1, rej_mask2

In [83]:
%%time
rej_mask, rej1, rej2 = percentile_clip(flat_cube, plow=0.001, phigh=0.001)
flat_cube_ma = np.ma.masked_array(flat_cube, mask=rej_mask)
master_flat = flat_cube_ma.mean(axis=0).filled()

plow = 0.001 ; phigh = 0.001
Wall time: 6.37 s


In [84]:
print('------master Flat (unclipped)-----')
astro.stats(master_flat)

------master Flat (unclipped)-----
count_px = 16394976
mean = 44202.589
median = 44358.436
avgDev = 2451.987
MAD = 2285.439
minimum = 51.796
maximum = 64736.152


In [85]:
rej1s = rej1.sum(axis=0)
rej2s = rej2.sum(axis=0)
nlows = round(rej1s.sum() / nframes)
nhighs = round(rej2s.sum() / nframes)
npixels = nx * ny
low_rate = nlows / npixels * 100
high_rate = nhighs / npixels * 100

In [88]:
rej1s.max(), rej2s.max(), rej2s.mean(), np.median(rej2s)

(5, 5, 4.340235020777096, 5.0)

In [82]:
rej1s.max(), rej2s.max(), rej2s.mean(), np.median(rej2s)

(5, 5, 0.201359916598841, 0.0)

In [112]:
vmin = 0
# vmax = 1
vmax = 5

plt.close('all')
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(9, 5))
axs[0,0].set_position([0.0, 0.4, 0.5, 0.5])
axs[0,1].set_position([0.5, 0.4, 0.5, 0.5])

axs[0,0].imshow(rej1s, vmin=vmin, vmax=vmax, cmap='gray')
axs[0,0].tick_params(labelbottom=False, labelleft=False)    
# axs[0].axis('off')
axs[0,0].text(70, 200, f'mean rejected pixels per image: {nlows} ({low_rate:.1f}%)', color='black',
        bbox=dict(facecolor='white', edgecolor='black', boxstyle='square', alpha=0.7))
axs[0,1].imshow(rej2s, vmin=vmin, vmax=vmax, cmap='gray')

axs[0,1].tick_params(labelbottom=False, labelleft=False)    
# axs[1].axis('off')
axs[0,1].text(70, 200, f'mean rejected pixels per image: {nhighs} ({high_rate:.1f}%)', color='black',
        bbox=dict(facecolor='white', edgecolor='black', boxstyle='square', alpha=0.7))

axs[1, 0].set_position([0.08, 0.07, 0.2, 0.2])
axs[1, 1].set_position([0.53, 0.07, 0.2, 0.2])
ax = fig.add_axes([0.3, 0.07, 0.2, 0.2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …